In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense

# Set random seed for reproducibility
np.random.seed(21)
random.seed(21)
tf.random.set_seed(21)

In [14]:
# Read in the independent and target variable datasets

stock = 'HPQ'
window_size = 20
pred_horizon = 10

X = pd.read_parquet(stock + '_X_' + str(window_size) + 'D.gzip')
y = pd.read_parquet(stock + '_y_' + str(pred_horizon) + 'D.gzip')
y = y.cumsum(axis=1)
y = y.iloc[:, -1:]
y_binary = (y > 0).astype(int)

In [15]:
# Split the dataset into training, validation and test datasets

num_features = 20

q_80 = int(len(X) * .8)
q_90 = int(len(X) * .9)

X_train, y_train = X[:q_80].to_numpy(), y_binary[:q_80].to_numpy()
X_val, y_val = X[q_80:q_90].to_numpy(), y_binary[q_80:q_90].to_numpy()
X_test, y_test = X[q_90:].to_numpy(), y_binary[q_90:].to_numpy()

X_train = X_train.reshape((-1, window_size, num_features))
X_val = X_val.reshape((-1, window_size, num_features))
X_test = X_test.reshape((-1, window_size, num_features))

In [16]:
# Define the early stopping callback to be used in all neural networks

early_stop = EarlyStopping(
    monitor='val_accuracy',        
    patience=5,                # wait 5 epochs for improvement
    restore_best_weights=True  # roll back to best weights
)

# Calculate class weights from training data

y_train_1D = y_train.reshape(-1)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_1D),
    y=y_train_1D
)
class_weights = dict(enumerate(class_weights))

In [17]:
# Fit an RNN model to the dataset

model_RNN = Sequential([
    SimpleRNN(16, 
         input_shape=(window_size, num_features)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_RNN.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001, rho=0.9),
              metrics=['accuracy'])

model_RNN.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, class_weight=class_weights, callbacks=[early_stop])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5150 - loss: 0.6922 - val_accuracy: 0.5141 - val_loss: 0.6948
Epoch 2/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5275 - loss: 0.6914 - val_accuracy: 0.5141 - val_loss: 0.6953
Epoch 3/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5262 - loss: 0.6909 - val_accuracy: 0.5141 - val_loss: 0.6952
Epoch 4/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5332 - loss: 0.6902 - val_accuracy: 0.5141 - val_loss: 0.6958
Epoch 5/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5369 - loss: 0.6897 - val_accuracy: 0.5141 - val_loss: 0.6967
Epoch 6/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5408 - loss: 0.6891 - val_accuracy: 0.5141 - val_loss: 0.6977


In [18]:
# Fit an LSTM model to the dataset

model_LSTM = Sequential([
    LSTM(16, 
         input_shape=(window_size, num_features)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_LSTM.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001, rho=0.9),
              metrics=['accuracy'])

model_LSTM.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, class_weight=class_weights, callbacks=[early_stop])

Epoch 1/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.5116 - loss: 0.6929 - val_accuracy: 0.5075 - val_loss: 0.6924
Epoch 2/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5205 - loss: 0.6916 - val_accuracy: 0.5141 - val_loss: 0.6918
Epoch 3/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5216 - loss: 0.6914 - val_accuracy: 0.5141 - val_loss: 0.6922
Epoch 4/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5193 - loss: 0.6911 - val_accuracy: 0.5141 - val_loss: 0.6936
Epoch 5/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5205 - loss: 0.6909 - val_accuracy: 0.5141 - val_loss: 0.6960
Epoch 6/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5271 - loss: 0.6906 - val_accuracy: 0.5141 - val_loss: 0.6979
Epoch 7/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5348 - loss: 0.6901 - val_accuracy: 0.5141 - val_loss: 0.7006


In [19]:
# Fit a GRU model to the dataset

model_GRU = Sequential([
    GRU(16, 
         input_shape=(window_size, num_features)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_GRU.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001, rho=0.9),
              metrics=['accuracy'])

model_GRU.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, class_weight=class_weights, callbacks=[early_stop])

Epoch 1/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.5092 - loss: 0.6952 - val_accuracy: 0.5141 - val_loss: 0.7288
Epoch 2/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.5172 - loss: 0.6929 - val_accuracy: 0.5141 - val_loss: 0.7280
Epoch 3/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.5224 - loss: 0.6925 - val_accuracy: 0.5141 - val_loss: 0.7282
Epoch 4/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.5229 - loss: 0.6924 - val_accuracy: 0.5141 - val_loss: 0.7203
Epoch 5/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5244 - loss: 0.6920 - val_accuracy: 0.5141 - val_loss: 0.7134
Epoch 6/100
302/302 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5265 - loss: 0.6918 - val_accuracy: 0.5141 - val_loss: 0.7154


In [20]:
def evaluate_classifier(y_true, y_probs, model_name='Model', threshold=0.5):

    # Generate binary predictions based on threshold
    y_pred = (y_probs > threshold).astype(int)

    # Classification report
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    # AUC
    auc = roc_auc_score(y_true, y_probs)

    # Build a summary row
    metrics = {
        "Model": model_name,
        "Accuracy": report["accuracy"],
        "Precision (0)": report["0"]["precision"],
        "Recall (0)": report["0"]["recall"],
        "F1 (0)": report["0"]["f1-score"],
        "Precision (1)": report["1"]["precision"],
        "Recall (1)": report["1"]["recall"],
        "F1 (1)": report["1"]["f1-score"],
        "AUC": auc
    }

    df_metrics = pd.DataFrame([metrics])
    df_metrics = df_metrics.apply(lambda x: round(x, 4))
    return df_metrics

In [21]:
# Predict the probabilities for the test dataset using the RNN model

y_RNN = model_RNN.predict(X_test)

fpr_RNN, tpr_RNN, thresholds_RNN = roc_curve(y_test, y_RNN)

# Choose threshold that gives better trade-off
J_RNN = tpr_RNN - fpr_RNN
optimal_idx_RNN = np.argmax(J_RNN)
optimal_threshold_RNN = thresholds_RNN[optimal_idx_RNN]

# Recalculate predictions
y_RNN_opt = (y_RNN > optimal_threshold_RNN).astype(int)

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [22]:
# Predict the probabilities for the test dataset using the LSTM model

y_LSTM = model_LSTM.predict(X_test)

fpr_LSTM, tpr_LSTM, thresholds_LSTM = roc_curve(y_test, y_LSTM)

# Choose threshold that gives better trade-off
J_LSTM = tpr_LSTM - fpr_LSTM
optimal_idx_LSTM = np.argmax(J_LSTM)
optimal_threshold_LSTM = thresholds_LSTM[optimal_idx_LSTM]

# Recalculate predictions
y_LSTM_opt = (y_LSTM > optimal_threshold_LSTM).astype(int)

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


In [23]:
# Predict the probabilities for the test dataset using the GRU model

y_GRU = model_GRU.predict(X_test)

fpr_GRU, tpr_GRU, thresholds_GRU = roc_curve(y_test, y_GRU)

# Choose threshold that gives better trade-off
J_GRU = tpr_GRU - fpr_GRU
optimal_idx_GRU = np.argmax(J_GRU)
optimal_threshold_GRU = thresholds_GRU[optimal_idx_GRU]

# Recalculate predictions
y_GRU_opt = (y_GRU > optimal_threshold_GRU).astype(int)

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [24]:
results_RNN = evaluate_classifier(y_test, y_RNN_opt, model_name='RNN', threshold=optimal_threshold_RNN)
results_LSTM = evaluate_classifier(y_test, y_LSTM_opt, model_name='LSTM', threshold=optimal_threshold_LSTM)
results_GRU = evaluate_classifier(y_test, y_GRU_opt, model_name='GRU', threshold=optimal_threshold_GRU)

all_results = pd.concat([results_RNN, results_LSTM, results_GRU])
display(all_results)


,Model,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),AUC
0,RNN,0.4332,0.4303,0.9903,0.5999,0.6667,0.0145,0.0284,0.5024
0,LSTM,0.4581,0.4413,0.9884,0.6101,0.8723,0.0596,0.1116,0.5240
0,GRU,0.4631,0.4404,0.9284,0.5974,0.6783,0.1134,0.1943,0.5209
